# **E1. ACTIVIDAD INTEGRADORA 1**

Rorigo Lopez Murguia

Victor Misael Escalante Alvarado



## Dependencias del proyecto

Funciones utilizadas para el funcionamiento del proyecto (Es necesario ejecutar antes de las siguientes celdas)

In [3]:
#Funciones del proyecto

class Protein:
    def __init__(self, name, sequence):
        self.name = name
        self.sequence = sequence

    def __str__(self):
        return f"{self.name}: {self.sequence}"

#Z array para string-matching
def ZArray(texto, patron):
    # Z - Array Algorithm
    n = len(texto)
    m = len(patron)

    L = 0
    R = 0
    x = n + m + 1
    Z = [-1 for i in range(x)]
    C = patron + "$" + texto

    for i in range(x):
        if i <= R:
            k = i - L
            if Z[k] < R - i + 1:
                Z[i] = Z[k]
            else:
                L = i
                while R < x and C[R-L] == C[R]:
                    R += 1
                Z[i] = R - L
                R -= 1
        else:
            L = R = i
            while R < x and C[R-L] == C[R]:
                R += 1
            Z[i] = R - L
            R -= 1
    return Z

def StringMatchZ(texto, patron):
  m = len(texto)
  n = len(patron)
  m = m + n + 1
  indexes = []
  zArray = ZArray(texto,patron)
  for i in range(m):
    if zArray[i] == n:
      # return i - n - 1
      indexes.append(i-n-1)
  return indexes

#Manacher para palíndromo más largo en los genes
def Manacher(frase):
    centro = limite = 0

    texto = "@$" + "$".join(frase) + "$#"
    e = len(texto)

    P = [0] * e

    for i in range(1, e - 1):
        if i < limite:
            simetrica = 2 * centro - i
            P[i] = min(limite - i, P[simetrica])

        while (i + P[i] + 1 < e and
               i - P[i] - 1 >= 0 and
               texto[i + P[i] + 1] == texto[i - P[i] - 1]):
            P[i] += 1

        if i + P[i] > limite:
            limite = i + P[i]
            centro = i

    maxVal = max(P)
    maxIndex = P.index(maxVal)
    inicio = (maxIndex - maxVal) // 2
    fin = inicio + maxVal
    return frase[inicio:fin], inicio, fin

#Trie Trees para búsqeuda de aminoácidos en las proteinas
class Nodo:
    def __init__(self, valor):
        self.valor = valor
        self.isFinal = False
        self.aminoacid = None
        self.hijos = []

    def agregar_hijo(self, valor):
        nuevo_nodo = Nodo(valor)
        self.hijos.append(nuevo_nodo)
        return nuevo_nodo

    def search_hijo(self, valor):
        for hijo in self.hijos:
            if hijo.valor == valor:
                return hijo
        return None

    def __str__(self):
        return str(self.valor)

class Trie:
    def __init__(self):
        self.raiz = Nodo("#")

    def agregar_palabra(self, palabra, aminoacid):
        actual = self.raiz
        for letra in palabra:
            hijo = actual.search_hijo(letra)
            if hijo is None:
                hijo = actual.agregar_hijo(letra)
            actual = hijo
        actual.isFinal = True
        actual.aminoacid = aminoacid

    def mostrar_arbol(self):
        actual = self.raiz
        queue = deque([actual])

        while queue:
            for _ in range(len(queue)):
                nodo = queue.popleft()
                print(nodo, end=" ")
                for hijo in nodo.hijos:
                    queue.append(hijo)
            print()

    def buscar_palabra(self, palabra):
        actual = self.raiz
        for letra in palabra:
            hijo = actual.search_hijo(letra)
            if hijo is None:
                return None
            else:
                actual = hijo
        if actual.isFinal:
            return actual.aminoacid
        else:
            #Imprime las palabras posibles
            print("Autocompletado para " + palabra + ":")
            palabras = buscar_palabras(actual, palabra)
            for palabra in palabras:
                if palabra == palabras[-1]:
                    print(palabra)
                else:
                  print(palabra, end=", ")
            return palabras

def buscar_palabras(nodo, palabra):
    palabras = []
    if nodo.isFinal:
        palabras.append(palabra)
    for hijo in nodo.hijos:
        palabras += buscar_palabras(hijo, palabra + hijo.valor)
    return palabras

# Buscar proteinas en secuencia
def SearchProteinas(sequence):
  lista_aminoacidos = []

  # Buscar todos los codones de inicio
  z = StringMatchZ(sequence, "ATG")

  # Por cada inicio de proteina, traducir hasta encontrar un STOP o hasta que acabe
  while len(z) > 0:
      start_index = z[0]
      end_index = z[0]

      protein = ""
      for i in range(start_index, len(sequence), 3): # Itera de 3 en 3 para buscar los codones
        codon = sequence[i:i+3]
        if len(codon) == 3:
          aminoacid = diccionario_aminoacidos.buscar_palabra(codon)
          if aminoacid == "STOP":
            break
          protein += aminoacid
          end_index = i+3

      lista_aminoacidos.append((z[0], protein))

      # Evita buscar proteinas en secciones que ya pertenecen a una proteina
      while z[0] < end_index and len(z) > 0:
        z.pop(0)
        if len(z) == 0:
          break

  results = []

  # Buscar las proteinas en las listas encontradas
  for aminoacid in lista_aminoacidos:
    for protein in proteins:
      z = StringMatchZ(aminoacid[1], protein.sequence)
      if len(z) > 0:
        for zz in z:
          results.append((protein.name, zz+aminoacid[0], len(protein.sequence)))
          # results.append((protein.name, aminoacid[0] ))
          print(protein.name, "encontrado en indice", str(zz+aminoacid[0]))
          print("Longitud en la secuencia:", len(protein.sequence))
          print("Codones:", sequence[zz+aminoacid[0]:zz+aminoacid[0]+12])
          print("Aminoácidos:" , end=" ")

          for x in protein.sequence[0:3]:
            print(amino_acid_dict[x], end=", ")
          print(amino_acid_dict[protein.sequence[3]])
          print()
  return results


#Creación de diccionario Trie para búsqueda de aminoácidos

aminoacids = [
    # Alanine
    ("GCT", "A"), ("GCC", "A"), ("GCA", "A"), ("GCG", "A"),
    # Arginine
    ("CGT", "R"), ("CGC", "R"), ("CGA", "R"), ("CGG", "R"), ("AGA", "R"), ("AGG", "R"),
    # Asparagine
    ("AAT", "N"), ("AAC", "N"),
    # Aspartic Acid
    ("GAT", "D"), ("GAC", "D"),
    # Cysteine
    ("TGT", "C"), ("TGC", "C"),
    # Glutamic Acid
    ("GAA", "E"), ("GAG", "E"),
    # Glutamine
    ("CAA", "Q"), ("CAG", "Q"),
    # Glycine
    ("GGT", "G"), ("GGC", "G"), ("GGA", "G"), ("GGG", "G"),
    # Histidine
    ("CAT", "H"), ("CAC", "H"),
    # Isoleucine
    ("ATT", "I"), ("ATC", "I"), ("ATA", "I"),
    # Leucine
    ("CTT", "L"), ("CTC", "L"), ("CTA", "L"), ("CTG", "L"), ("TTA", "L"), ("TTG", "L"),
    # Lysine
    ("AAA", "K"), ("AAG", "K"),
    # Methionine (Start Codon)
    ("ATG", "M"),
    # Phenylalanine
    ("TTT", "F"), ("TTC", "F"),
    # Proline
    ("CCT", "P"), ("CCC", "P"), ("CCA", "P"), ("CCG", "P"),
    # Serine
    ("TCT", "S"), ("TCC", "S"), ("TCA", "S"), ("TCG", "S"), ("AGT", "S"), ("AGC", "S"),
    # Threonine
    ("ACT", "T"), ("ACC", "T"), ("ACA", "T"), ("ACG", "T"),
    # Tryptophan
    ("TGG", "W"),
    # Tyrosine
    ("TAT", "Y"), ("TAC", "Y"),
    # Valine
    ("GTT", "V"), ("GTC", "V"), ("GTA", "V"), ("GTG", "V"),
    # Stop Codons
    ("TAA", "STOP"), ("TAG", "STOP"), ("TGA", "STOP")
]

amino_acid_dict = {
    'A': 'Alanine',
    'R': 'Arginine',
    'N': 'Asparagine',
    'D': 'Aspartic Acid',
    'C': 'Cysteine',
    'E': 'Glutamic Acid',
    'Q': 'Glutamine',
    'G': 'Glycine',
    'H': 'Histidine',
    'I': 'Isoleucine',
    'L': 'Leucine',
    'K': 'Lysine',
    'M': 'Methionine',
    'F': 'Phenylalanine',
    'P': 'Proline',
    'S': 'Serine',
    'T': 'Threonine',
    'W': 'Tryptophan',
    'Y': 'Tyrosine',
    'V': 'Valine',
    'STOP': 'Stop Codon'
}


diccionario_aminoacidos = Trie()

for aminoacido in aminoacids:
    diccionario_aminoacidos.agregar_palabra(aminoacido[0], aminoacido[1])


# Lectura de archivos

In [4]:
# Leer archivos

with open('SARS-COV-2-MN908947.3.txt', 'r') as file:
    lines = file.readlines()
    secuencia_wuhan = ""
    for line in lines[1:]:
        line = line.strip()
        secuencia_wuhan += line

with open('SARS-COV-2-MT106054.1.txt', 'r') as file:
    lines = file.readlines()
    secuencia_texas = ""
    for line in lines[1:]:
        line = line.strip()
        secuencia_texas += line

with open('gen-M.txt', 'r') as file:
    lines = file.readlines()
    gen_m = ""
    for line in lines[1:]:
        line = line.strip()
        gen_m += line

with open('gen-S.txt', 'r') as file:
    lines = file.readlines()
    gen_s = ""
    for line in lines[1:]:
        line = line.strip()
        gen_s += line

with open('gen-ORF1AB.txt', 'r') as file:
    lines = file.readlines()
    gen_o = ""
    for line in lines[1:]:
        line = line.strip()
        gen_o += line

with open('seq-proteins.txt', 'r') as file:
    lines = file.readlines()
    proteins = []
    for line in lines:
        if line.startswith(">"):
            name = line.strip()[1:]
            sequence = ""
        else:
            sequence += line.strip()
            proteins.append(Protein(name, sequence))

FileNotFoundError: [Errno 2] No such file or directory: 'SARS-COV-2-MN908947.3.txt'

# 1. Aparicion de Los genes

En la siguiente seccion se obtienen los **indices** de aparicion de los genes en la secuencia del virus *SARS-COV-2-MN908947.3*

**Complejidades**

Para la búsqueda de las cadenas de los genes en la cadena de Wuhan (2019) se utilizó string matching con Z-Array, que tiene una complejidad de O(n), donde en este caso, n es la cantidad de aminoácidos en la secuencia. El procedimiento se realiza una vez por cada gen a buscar.

In [ ]:
# Indices de aparición de los genes en la secuencia del virus SARS-COV-2-MN908947.3.txt

print("Apariciones del gen M: ")
gen_m_indices = StringMatchZ(secuencia_wuhan, gen_m)
print(gen_m_indices)
print(secuencia_wuhan[gen_m_indices[0]:gen_m_indices[0]+12])

print("Apariciones del gen S: ")
gen_s_indices = StringMatchZ(secuencia_wuhan, gen_s)
print(gen_s_indices)
print(secuencia_wuhan[gen_s_indices[0]:gen_s_indices[0]+12])

print("Apariciones del gen ORF1AB: ")
gen_o_indices = StringMatchZ(secuencia_wuhan, gen_o)
print(gen_o_indices)
print(secuencia_wuhan[gen_o_indices[0]:gen_o_indices[0]+12])

Apariciones del gen M: 
[26522]
ATGGCAGATTCC
Apariciones del gen S: 
[21562]
ATGTTTGTTTTT
Apariciones del gen ORF1AB: 
[265]
ATGGAGAGCCTT


# 2. Palindromo Mas largo

En la siguiente seccion se muestran los palindromos mas largos para cada uno de los genomas


---

**Complejidades**
Para encontrar palíndromos, se utilizó el algorítmo de Manacher, que cuenta con una complejidad O(n) donde n es la cantidad de aminoácidos en la secuencia.

In [ ]:
#Palíndromo más largo en cada uno de los tres genes

print("Palíndromo más grande para gen M")
palindrome_m, inicio_m, fin_m = Manacher(gen_m)
print(palindrome_m, "Longitud:", len(palindrome_m))
print('\n')

print("Palíndromo más grande para gen S")
palindrome_s, inicio_s, fin_s = Manacher(gen_s)
print(palindrome_s, "Longitud:", len(palindrome_s))
print('\n')

print("Palíndromo más grande para gen ORF1AB")
palindrome_o, inicio_o, fin_o = Manacher(gen_o)
print(palindrome_o, "Longitud:", len(palindrome_o))
print('\n')

# Escribir palindromos en archivo
with open('palindromos.txt', 'w') as file:
  file.write("M\n"+str(inicio_m)+","+str(fin_m)+","+palindrome_m+"\n")
  file.write("S\n"+str(inicio_s)+","+str(fin_s)+","+palindrome_s+"\n")
  file.write("ORF1AB\n"+str(inicio_o)+","+str(fin_o)+","+palindrome_o+"\n")
  file.close()

Palíndromo más grande para gen M
CTAAAGAAATC Longitud: 11
Palíndromo más grande para gen S
GAATTTGTGTTTAAG Longitud: 15
Palíndromo más grande para gen ORF1AB
CTCAATGACTTCAGTAACTC Longitud: 20


# 3. Busqueda de Proteinas

En este apartado mostramos las apariciones de las proteinas de "seq-proteins.txt" en la secuencia del virus de de Wuhan (2019)

Se muestra el nombre de la proteína, el índice donde se comienza a producir, sus primeros aminoácidos y sus codones asociados

---

**Estructuras**

Para relacionar trios de aminoácidos con codones, se utilizó un diccionario Trie que ayuda a almacenar y encontrar las diferentes posibilidades de codones en una sola estructura.

**Procedimiento**

Se utilizó String Matching para encontrar todas las apariciones del codón START (M) en la secuencia. A partir de ahí, se leen los codones de 3 en 3 aminoácidos hasta encontrar un codón STOP o acabar la secuencia. Si dentro de una proteína se encuentran otros codones START, no se toman en cuenta para la traducción de una nueva proteina, solo se trata como un codón más dentro de la misma traducción.

**Complejidad**

 El procedimiento completo tiene una complejidad en el dominio de O(n), donde n es la cantidad de aminoácidos en la secuencia.

In [ ]:
print("Proteinas Wuhan")
proteinas_wuhan = SearchProteinas(secuencia_wuhan)

print(proteinas_wuhan)

Proteinas Wuhan
QHD43415_1 encontrado en indice 265
Longitud en la secuencia: 180
Codones: ATGGAGAGCCTT
Aminoácidos: Methionine, Glutamic Acid, Serine, Leucine

QHD43415_2 encontrado en indice 445
Longitud en la secuencia: 638
Codones: TTGCCTCAACTT
Aminoácidos: Alanine, Tyrosine, Threonine, Arginine

QHD43415_3 encontrado en indice 1083
Longitud en la secuencia: 1945
Codones: TGTATTTCCCTT
Aminoácidos: Alanine, Proline, Threonine, Lysine

QHD43415_4 encontrado en indice 3028
Longitud en la secuencia: 500
Codones: TGTTCTTTCTAC
Aminoácidos: Lysine, Isoleucine, Valine, Asparagine

QHD43415_5 encontrado en indice 3528
Longitud en la secuencia: 306
Codones: TGATTACATAGC
Aminoácidos: Serine, Glycine, Phenylalanine, Arginine

QHD43415_6 encontrado en indice 3834
Longitud en la secuencia: 290
Codones: TTTGGAAATGAA
Aminoácidos: Serine, Alanine, Valine, Lysine

QHD43415_7 encontrado en indice 4124
Longitud en la secuencia: 83
Codones: TGGGTGATGTTG
Aminoácidos: Serine, Lysine, Methionine, Serine



# 4. Comparar proteinas entre Wuhan (2019) y Texas (2020)

Se realiza el mismo procedimiento que en el punto numero 3 con la secuencia de Texas, se encuentran y muestran las diferencias encontradas.

**Complejidad**

Para encontrar las proteinas en la secuencia de Texas (2020) se realiza el mismo procedimiento que en el punto 3, que tiene una complejidad de O(n), donde n es la cantidad de aminoácidos en la secuencia.

Para calcular las diferencias entre ambas secuencias, se comparan los resultados de las búsquedas de las proteínas. Este procedimiento tiene una complejidad de O(n), donde n es la cantidad de proteinas encontradas en la secuencia.

In [ ]:
print("Proteinas Texas")
proteinas_texas = SearchProteinas(secuencia_texas)

print(proteinas_texas)

diferencias_extra_wuhan = [item for item in proteinas_wuhan if item not in proteinas_texas]
diferencias_extra_texas = [item for item in proteinas_texas if item not in proteinas_wuhan]

print()
print("Proteínas extra en Wuhan 2019")
print(diferencias_extra_wuhan)
print()
print("Proteínas extra en Texas 2020")
print(diferencias_extra_texas)

Proteinas Texas
QHD43415_1 encontrado en indice 265
Codones: ATGGAGAGCCTT
Aminoácidos: Methionine, Glutamic Acid, Serine, Leucine

QHD43415_2 encontrado en indice 445
Codones: TTGCCTCAACTT
Aminoácidos: Alanine, Tyrosine, Threonine, Arginine

QHD43415_3 encontrado en indice 1083
Codones: TGTATTTCCCTT
Aminoácidos: Alanine, Proline, Threonine, Lysine

QHD43415_4 encontrado en indice 3028
Codones: TGTTCTTTCTAC
Aminoácidos: Lysine, Isoleucine, Valine, Asparagine

QHD43415_5 encontrado en indice 3528
Codones: TGATTACATAGC
Aminoácidos: Serine, Glycine, Phenylalanine, Arginine

QHD43415_6 encontrado en indice 3834
Codones: TTTGGAAATGAA
Aminoácidos: Serine, Alanine, Valine, Lysine

QHD43415_7 encontrado en indice 4124
Codones: TGGGTGATGTTG
Aminoácidos: Serine, Lysine, Methionine, Serine

QHD43415_8 encontrado en indice 4207
Codones: AAAGCTTTGAGA
Aminoácidos: Alanine, Isoleucine, Alanine, Serine

QHD43415_9 encontrado en indice 4405
Codones: CATGCAGAAGAA
Aminoácidos: Asparagine, Asparagine, Glut

# 4.B (Extra) Comparar Secciones de las Secuencias

Esta sección se utilizó para pruebas en la sección 4

In [ ]:
#Ver las diferencias en los aminoacidos


start = 265
length = 12

comp1 = secuencia_wuhan[start:start+length]
comp2 = secuencia_texas[start:start+length]

print(comp1 == comp2)

print(comp1)
print(comp2)


min_len = min(len(comp1), len(comp2))
comp1 = comp1[:min_len]
comp2 = comp2[:min_len]

# Identificar posiciones donde las secuencias difieren
defff = [(i, comp1[i], comp2[i]) for i in range(min_len) if comp1[i] != comp2[i]]

# Imprimir las posiciones y diferencias
print("Posiciones donde las secuencias difieren:")
for diff in defff:
    print(f"Posición {diff[0]}: {diff[1]} (secuencia1) != {diff[2]} (secuencia2)")


True
ATGGAGAGCCTT
ATGGAGAGCCTT
Posiciones donde las secuencias difieren:
